<a href="https://colab.research.google.com/github/tarikaytek/tensorflow-developer-professional-certificate/blob/main/C3W2_sarcarsm-classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2022-10-31 21:13:30--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.119.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2022-10-31 21:13:30 (192 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]



In [3]:
import json

with open("./sarcasm.json", 'r') as f:
  datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])

len(sentences)

26709

In [4]:
training_size = 2000
vocab_size = 10000
max_length = 32
embedding_dim = 16

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]

training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<UNK>")

tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

In [6]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 16)            160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [8]:
num_epochs = 30

history = model.fit(training_padded, training_labels, epochs=num_epochs,
                    validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
63/63 - 4s - loss: 0.6889 - accuracy: 0.5470 - val_loss: 0.6845 - val_accuracy: 0.5613 - 4s/epoch - 66ms/step
Epoch 2/30
63/63 - 3s - loss: 0.6794 - accuracy: 0.5580 - val_loss: 0.6755 - val_accuracy: 0.5613 - 3s/epoch - 44ms/step
Epoch 3/30
63/63 - 3s - loss: 0.6596 - accuracy: 0.5765 - val_loss: 0.6557 - val_accuracy: 0.5952 - 3s/epoch - 44ms/step
Epoch 4/30
63/63 - 1s - loss: 0.6137 - accuracy: 0.7600 - val_loss: 0.6152 - val_accuracy: 0.7195 - 1s/epoch - 23ms/step
Epoch 5/30
63/63 - 3s - loss: 0.5329 - accuracy: 0.8805 - val_loss: 0.5597 - val_accuracy: 0.7882 - 3s/epoch - 44ms/step
Epoch 6/30
63/63 - 3s - loss: 0.4338 - accuracy: 0.9265 - val_loss: 0.5052 - val_accuracy: 0.7969 - 3s/epoch - 44ms/step
Epoch 7/30
63/63 - 3s - loss: 0.3387 - accuracy: 0.9425 - val_loss: 0.4664 - val_accuracy: 0.7996 - 3s/epoch - 44ms/step
Epoch 8/30
63/63 - 1s - loss: 0.2651 - accuracy: 0.9560 - val_loss: 0.4427 - val_accuracy: 0.8024 - 1s/epoch - 23ms/step
Epoch 9/30
63/63 - 1s - loss: 0.